In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time
# Main slim library
slim = tf.contrib.slim
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/marcotcr/slim/models/slim')
sys.path.append('/home/marcotcr/phd/comparing_explanations/python')

In [ ]:
from nets import inception
from preprocessing import inception_preprocessing
import anchor

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
session = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
image_size = inception.inception_v3.default_image_size
def transform_img_fn(path_list):
    out = []
    for f in path_list:
        image_raw = tf.image.decode_jpeg(open(f).read(), channels=3)
        image = inception_preprocessing.preprocess_image(image_raw, image_size, image_size, is_training=False)
        out.append(image)
    return session.run([out])[0]

In [ ]:
from datasets import imagenet
names = imagenet.create_readable_names_for_imagenet_labels()


In [ ]:
# image = tf.image.decode_jpeg(open('maxresdefault.jpg').read(), channels=3)
# processed_image = inception_preprocessing.preprocess_image(image, image_size, image_size, is_training=False)
# processed_images  = tf.expand_dims(processed_image, 0)


In [ ]:
processed_images = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))

In [ ]:
import os
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, _ = inception.inception_v3(processed_images, num_classes=1001, is_training=False)
probabilities = tf.nn.softmax(logits)

checkpoints_dir = '/home/marcotcr/phd/comparing_explanations/notebooks/inception_tf'
init_fn = slim.assign_from_checkpoint_fn(
    os.path.join(checkpoints_dir, 'inception_v3.ckpt'),
    slim.get_model_variables('InceptionV3'))
init_fn(session)


In [ ]:
def predict_fn(images):
    return session.run(probabilities, feed_dict={processed_images: images})

In [ ]:
# def predict_fn(images):
#     with slim.arg_scope(inception.inception_v3_arg_scope()):
#         logits, _ = inception.inception_v3(images, num_classes=1001, is_training=False, reuse=True)
#     probabilities = tf.nn.softmax(logits)
#     return session.run([probabilities])[0]

In [ ]:
import copy
from skimage.segmentation import quickshift, mark_boundaries, slic, felzenszwalb
def ShowImageNoAxis(image, boundaries=None, save=None):
    fig = plt.figure()
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    if boundaries is not None:
        ax.imshow(mark_boundaries(image / 2 + 0.5, boundaries))
    else:
        ax.imshow(image / 2 + .5)
    if save is not None:
        plt.savefig(save)
    plt.show()

def show_exp(segments, exp, image, explainer):
    mask = np.ones(segments.shape).astype(bool)
    temp = copy.deepcopy(image)
    temp_img = copy.deepcopy(temp)
#     temp.img = temp.fudged_image.copy()
    temp[:] = 0
    # for x in exp_greedy[:3]:
    #     x = (x, x)
    for x in exp:
        temp[segments == x[0]] = temp_img[segments==x[0]]
    # temp.img[mask] = np.random.random(mask.nonzero()[0].shape[0] * 3).reshape(mask.nonzero()[0].shape[0], 3)
    print 'Anchor for prediction ', names[predict_fn(np.expand_dims(image, 0))[0].argmax()], 'confidence', exp[-1][2]
    ShowImageNoAxis(temp)
    print 'Counter Examples:'
    for e in exp[-1][3]:
        data = e[:-1]
        temp = explainer.dummys[e[-1]].copy()
        for x in data.nonzero()[0]:
            temp[segments == x] = image[segments == x]
        ShowImageNoAxis(temp)
        print 'Prediction = ', names[predict_fn(np.expand_dims(temp, 0))[0].argmax()]

def show_fudged_exp(segments, exp, image):
    fudged_image = image.copy()
    for x in np.unique(segments):
        fudged_image[segments == x] = (np.mean(image[segments == x][:, 0]),
                                       np.mean(image[segments == x][:, 1]),
                                       np.mean(image[segments == x][:, 2]))
    mask = np.ones(segments.shape).astype(bool)
    temp = copy.deepcopy(image)
    temp_img = copy.deepcopy(temp)
#     temp.img = temp.fudged_image.copy()
    temp[:] = 0
    # for x in exp_greedy[:3]:
    #     x = (x, x)
    for x in exp:
        temp[segments == x[0]] = temp_img[segments==x[0]]
    # temp.img[mask] = np.random.random(mask.nonzero()[0].shape[0] * 3).reshape(mask.nonzero()[0].shape[0], 3)
    print 'Anchor for prediction ', names[predict_fn(np.expand_dims(image, 0))[0].argmax()], 'confidence', exp[-1][2]
    ShowImageNoAxis(temp)
    print 'Counter Examples:'
    for e in exp[-1][3]:
        data = e
        temp = fudged_image.copy()
        for x in data.nonzero()[0]:
            temp[segments == x] = image[segments == x]
        ShowImageNoAxis(temp)
        print 'Prediction = ', names[predict_fn(np.expand_dims(temp, 0))[0].argmax()]

In [ ]:
import skimage.io
import skimage.transform
def transform_img_fast(path):
    img = skimage.io.imread(path)
    if len(img.shape) != 3:
        img = skimage.color.gray2rgb(img)
    short_egde = min(img.shape[:2])
    yy = int((img.shape[0] - short_egde) / 2)
    xx = int((img.shape[1] - short_egde) / 2)
    crop_img = img[yy: yy + short_egde, xx: xx + short_egde]
    return (skimage.transform.resize(crop_img, (299, 299)) - 0.5) * 2
def transform_img_fn_fast(paths):
    out = []
    for i, path in enumerate(paths):
        if i % 100 == 0:
            print i
        out.append(transform_img_fast(path))
    return np.array(out)
#     return np.array([transform_img_fast(path) for path in paths])

In [ ]:
a = time.time()
images = transform_img_fn(['zebra.jpg', 'zebra.jpg'])
print time.time() - a


In [ ]:
img = images[-1]

In [ ]:
np.array(images).shape

In [ ]:
a = time.time()
probs = predict_fn(np.array(images))
print time.time() -a 

In [ ]:
probs

In [ ]:
probs.shape

In [ ]:
i = 1
ShowImageNoAxis(images[i])
names[np.argmax(probs[i])]

In [ ]:
image = images[0]

In [ ]:
3

In [ ]:
explainer = anchor.AnchorImage([], '/Users/marcotcr/phd/datasets/imagenet/full',
                               transform_img_fn=transform_img_fn_fast, n=5000)

In [ ]:
# explainer2 = anchor.AnchorImage([])


In [ ]:
# image = transform_img_fast('/home/marcotcr/phd/datasets/visualqa/val2014/COCO_val2014_000000045626.jpg')
image = transform_img_fast('nick.jpg')
ShowImageNoAxis(image)

In [ ]:
z = predict_fn([image])
print names[z.argmax()], z.max()

In [ ]:
from skimage.segmentation import quickshift
segments = quickshift(image, kernel_size=4, max_dist=200, ratio=0.2)
exp = {}
# exp['feature'] =  [1, 2, 7, 8, 9, 11, 16, 17, 24, 25, 28, 29, 30, 34, 36, 39, 40, 52]
exp['feature'] = [1, 2, 7, 8, 9, 11, 17, 25, 29, 30, 34, 36, 40, 52]
temp = np.ones(image.shape)
mask = np.zeros(segments.shape).astype(bool)
for f in exp['feature']:
    mask[segments == f] = 1
temp[mask] = image[mask].copy()
ShowImageNoAxis(temp, mask, '/tmp/nickz.png')

In [ ]:
N = 5
chosen = np.random.choice(len(explainer.dummys), N)
imgs = []
masks = []
for c in chosen:
    temp = copy.deepcopy(explainer.dummys[c])
    mask = np.zeros(segments.shape).astype(bool)
    for f in exp['feature']:
        mask[segments == f] = 1
    temp[mask] = image[mask].copy()
    imgs.append(temp)
    masks.append(mask)
preds = predict_fn(imgs)
for i, p in zip(imgs, preds):
    if p.max() < 0.9:
        continue
    ShowImageNoAxis(i, mask, '/tmp/nickz.png')
    print names[p.argmax()], p.max()


In [ ]:
(1, 2, 7, 8, 9, 11, 16, 17, 24, 25, 28, 29, 30, 34, 36, 39, 40, 52) mean = 0.97 lb = 0.92 ub = 0.99 coverage: 0.00

In [ ]:
# (1, 2, 7, 8, 9, 11, 17, 25, 29, 30, 34, 36, 40, 52) mean 0.95
# (1, 2, 7, 8, 9, 11, 16, 17, 25, 29, 30, 34, 36, 40, 52) mean = 0.90 lb = 0.87 ub = 0.93 coverage: 0.00
# (1, 2, 7, 8, 9, 11, 16, 17, 24, 25, 29, 30, 34, 36, 40, 52) mean = 0.94 lb = 0.90 ub = 0.97 coverage: 0.00
# (1, 2, 7, 8, 9, 11, 16, 17, 24, 25, 28, 29, 30, 34, 36, 39, 40, 52) mean = 0.97 lb = 0.92 ub = 0.99 coverage: 0.00


In [ ]:
segments, exp = explainer.explain_hoeffding(image, predict_fn, threshold=0.95, batch_size=10,
                                            tau=0.20, verbose=True, min_shared_samples=200, beam_size=1)
# segments_f, exp_f = explainer2.explain_hoeffding(image, predict_fn, threshold=0.95, batch_size=20,
#                                             tau=0.1, verbose=True, min_shared_samples=100, new=True)

In [ ]:
exp

In [ ]:
image2 = transform_img_fn(['zebra.jpg'])[0]
segments2, exp2 = explainer.explain_hoeffding(image2, predict_fn, threshold=0.95, batch_size=10,
                                            tau=0.15, verbose=False, min_shared_samples=200, beam_size=1)

In [ ]:
def random_examples_from_exp(segments, exp, image, explainer, n=5):
    n_features = len(np.unique(segments))
    data = np.random.choice(
        [0, 1], n * n_features, p=[.8, .2]).reshape(
        (n, n_features))
    present = np.array([x[0] for x in exp])
    data[:, present] = 1
    chosen = np.random.choice(range(len(explainer.dummys)), data.shape[0],
                              replace=True)
    imgs = []
    for d, r in zip(data, chosen):
        temp = copy.deepcopy(image)
        zeros = np.where(d == 0)[0]
        mask = np.zeros(segments.shape).astype(bool)
        for z in zeros:
            mask[segments == z] = True
        chosen = np.random
        temp[mask] = explainer.dummys[r][mask]
        imgs.append(temp)
    preds = predict_fn(np.array(imgs))
    preds_max = np.argmax(preds, axis=1)
    preds_val = np.max(preds, axis=1)
    for img, pred, val in zip(imgs, preds_max, preds_val):
        ShowImageNoAxis(img)
        print names[pred], val

In [ ]:
# show_fudged_exp(segments_f, exp_f, image)
show_exp(segments, exp, image, explainer)

In [ ]:
izz = random_examples_from_exp(segments, exp, image, explainer, n=10)

In [ ]:
ShowImageNoAxis(image)

In [ ]:
a = time.time()
samples = fn([], 20)
print time.time() - a

In [ ]:
explainer.

In [ ]:
6.05 / len(ret)

In [ ]:
[names[i] for i in probs.argmax(axis=1)]

In [ ]:
# import time
# image = tf.image.decode_jpeg(open('maxresdefault.jpg').read(), channels=3)
# processed_image = inception_preprocessing.preprocess_image(image, image_size, image_size, is_training=False)
# processed_images  = tf.expand_dims(processed_image, 0)
# images
a = time.time()
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, _ = inception.inception_v3(processed_images, num_classes=1001, is_training=False, reuse=True)
probabilities = tf.nn.softmax(logits)
np_image, probabilities = session.run([image, probabilities])
probabilities = probabilities[0, 0:]
print time.time() - a

In [ ]:
# images = processed_images.eval()
# images = np.vstack((images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images, images))
a = time.time()
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, _ = inception.inception_v3(images[:20], num_classes=1001, is_training=False, reuse=True)
probabilities = tf.nn.softmax(logits)
np_image, probabilities = session.run([image, probabilities])
# probabilities = probabilities[0, 0:]
print time.time() - a

In [ ]:
6.36 / 20.

In [ ]:
from skimage.segmentation import quickshift, mark_boundaries, slic, felzenszwalb
segments = quickshift(image, kernel_size=5, max_dist=200, ratio=0.5)
fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(mark_boundaries(image / 2. + .5,segments))
plt.show()

In [ ]:
np.unique(segments).shape

In [ ]:
plt.imshow(images[0] / 2. + 0.5)

In [ ]:
[names[np.argmax(i)] for i in probabilities]

In [ ]:
import os
checkpoints_dir = '/home/marcotcr/phd/comparing_explanations/notebooks/inception_tf'
init_fn = slim.assign_from_checkpoint_fn(
        os.path.join(checkpoints_dir, 'inception_v3.ckpt'),
        slim.get_model_variables('InceptionV3'))
init_fn(session)


In [ ]:
names[probabilities.argmax()]

In [ ]:
probabilities

In [ ]:
logits, _ = inception.inception_v3(processed_images, num_classes=1001, is_training=False)
probabilities = tf.nn.softmax(logits)

In [ ]:
images[0]

In [ ]:
plt.imshow(image.eval().astype(np.uint8))

In [ ]:
processed_image.eval()

In [ ]:
plt.imshow(processed_image.eval())

In [ ]:
plt.imshow(image)

In [ ]:
image.eval().shape

In [ ]:
model_dir = 'inception'

In [ ]:
image = os.path.join(model_dir, 'cropped_panda.jpg')

In [ ]:
image = 'doglion.jpg'
image_data = tf.gfile.FastGFile(image, 'rb').read()


In [ ]:
a = tf.cast(image_data, tf.float32)

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(a)

In [ ]:
image_data